### rapyuta.io SDK minio file-server example.
Deploy minio file server to rapyuta.io

ojb-store-deployment-tutorial: https://userdocs.rapyuta.io/developer-guide/create-software-packages/persistent-storage/obj-store-deployment-tutorial/

SDK doc: https://docs.rapyuta.io/python-sdk/

#### 1.Auth, Parameter and File psths.
you can get auth from rapyuta.io console

SDK auth token:https://userdocs.rapyuta.io/developer-guide/tooling-automation/python-sdk/sdk-tokens-parameters/

In [ ]:
AUTH_TOKEN = ''
PROJECT_ID = ''

#params
retry_count=90
interval=10
minio_params = {
    'MINIO_ACCESS_KEY': 'rapyuta123',
    'MINIO_SECRET_KEY': 'rapyuta123'
}

#file paths of package manifests
base_path = '../io_manifests/'
minio_manifest = 'minio_file_server_manifest.json'

#### 2. import module and create Client Instance

In [ ]:
from rapyuta_io import Client, DeploymentPhaseConstants, DiskType

client = Client(AUTH_TOKEN, PROJECT_ID)

#### 3. Create package

In [ ]:
import os
import json
import time
from pprint import pprint

def create_package(file_path, packages):
    json_data = open(file_path, 'r')
    json_load = json.load(json_data)
    package_id = None
    for package in packages:
        if package['packageName'] == json_load['name'] and  \
           package['packageVersion'] == json_load['packageVersion']:
            package_id = package['packageId']
            break
    if not package_id:
        res = client.create_package(json_load, retry_limit=3)
        print(json_load['name'] + json_load['packageVersion'] + ' was created')
        package_id = res['packageId']
    else :
        print(json_load['name'] + json_load['packageVersion'] + ' exist')
    
    return package_id

def get_plan_id(pakcage_id, index=0):
    pack = client.get_package(pakcage_id)
    return pack['plans'][index]['planId']

def get_ids(base_path, manifest, packages):
    file_path = os.path.join(base_path, manifest)
    package_id = create_package(file_path, packages)
    plan_id = get_plan_id(package_id) 
    return {
        'packageId': package_id,
        'planId': plan_id        
    }

packages = client.get_all_packages()
# pprint(packages)

minio_ids =    get_ids(base_path, minio_manifest     ,packages)    

#### 4.Create persistent volume

In [ ]:
volume = client.get_persistent_volume()
volumes = volume.get_all_volume_instances()

persistent_vol_deployment = None
persistent_vol_name = 'persistent_vol_minio'
for vol in volumes:
    if vol['name'] == persistent_vol_name and vol['phase'] == 'Succeeded':
        persistent_vol_deployment = volume.get_volume_instance(vol['deploymentId'])
        print('Persistent Vol was found and will be used.')
        break

if not persistent_vol_deployment: 
    print ('Creating a new persistent volume deployment... ')
    persistent_vol_deployment = volume.create_volume_instance(persistent_vol_name, 32, DiskType.SSD)
    result = persistent_vol_deployment.poll_deployment_till_ready(retry_count=retry_count, sleep_interval=interval)
    print ('Persistent Vol are created')

#### 3.Deploy package

In [ ]:
# Device Talker code snippet
package = client.get_package(minio_ids['packageId'])
configuration = package.get_provision_configuration(minio_ids['planId'])
for key in minio_params:
    configuration.add_parameter('MinIO_FS', key, minio_params[key])
configuration.mount_volume(component_name='MinIO_FS',
                           volume_instance=persistent_vol_deployment,
                           mount_path="/data")
deployment = package.provision(deployment_name = 'minio-file-server',
						provision_configuration = configuration)
result = deployment.poll_deployment_till_ready(retry_count=retry_count, sleep_interval=interval)
print 'Minio endpoint: ' + result['componentInfo'][0]['networkEndpoints']['FileStorage']

#### 4. Deprovison all

In [ ]:
deployment.deprovision()